In [ ]:
from collections import OrderedDict
import os
from datetime import datetime
import json

import numpy as np
import matplotlib.pyplot as plt

from scripts.utils import *
from scripts.models import *
from scripts.visualizations import *
from scripts.pruning import *
from scripts.activation_patterns import *
from scripts.stats import *
from scripts.activation_regions import *
from scripts.lp_decision_trees import *

## Hyperparams
...That you actually change

In [ ]:
notebook_name = 'visualizations'
test_name = 'example'

### Saving

In [ ]:
save_figures = False

results_from_training = False
visualize_AP_based = False

### Select test to visualize 

In [ ]:
results_root_dir = './results/'

In [ ]:
results_folders = get_folders(results_root_dir)
results_folders.sort()
for i, folder in enumerate(results_folders):
    print(i, folder)

### Which results should we visualize?

In [ ]:
result_ind = 0

In [ ]:
results_dir = results_root_dir + results_folders[result_ind] + '/'
files = get_files_in_dir(results_dir)

In [ ]:
data = config = hyperparams = metadata = None
for file in files:
    name, end = file.split('.')
    
    print('read', name, end)
    
    if name == 'data':
        data = torch.load(results_dir + file)
    elif name == 'hyperparams':
        hyperparams = torch.load(results_dir + file)
    elif name == 'config':
        with open(results_dir + file, 'rb') as fo:
            if end == 'json':
                config = json.loads(fo.read())
            else:
                config_b = fo.read()
    elif name == 'metadata':
        with open(results_dir + file, 'rb') as fo:
            if end == 'json':
                metadata = json.loads(fo.read())
            else:
                metadata_b = fo.read()
    else:
        print('unexpected file ', file)

In [ ]:
if hyperparams is None:
    
    def try_fetch_hyperparams(model_root):
        model_dir = model_root + results_folders[result_ind] + '/'
        print('try to fetch hyperparams from', model_dir)
        if not os.path.isdir(model_dir):
            print(f'\tdir {model_dir} didnt exist!')
            return None
        
        files = get_files_in_dir(model_dir)
        for file in files:
            if 'hyperparams' in file:
                hyperparams = torch.load(model_dir + file)
                print('Success!')
                
        return hyperparams
        
    model_root = '../mnist/models/'
    model_root = './models/'
    
    hyperparams = try_fetch_hyperparams(model_root)
    
print(hyperparams.keys())

In [ ]:
data.keys()

In [ ]:
for key in data:
    print(key, data[key].keys())

In [ ]:
data['ar_2d'].keys() if not results_from_training else data['ar_2d_training'].keys()

In [ ]:
print(metadata.keys())

In [ ]:
metadata

In [ ]:
if not results_from_training:
    print(config.keys())

config

## Parse experiment configuration

In [ ]:
def infer_metada_from_hyperparams(hyperparams):
    param_keys = OrderedDict()
    for name in hyperparams:
        param_keys[name] = [
            hyp[0][-1] for hyp in hyperparams[name]
        ]

    model_architectures = []
    for name in hyperparams:
        model_architectures.append(name)
    return param_keys, model_architectures

### Data

In [ ]:
use_training_setup = config['data'].get('use_training_setup', True) if not results_from_training else True

data_conf_str = 'training metadata' if use_training_setup else 'configuration file'
print(f'get data configuration from {data_conf_str}')

data_conf = config['data'] if not use_training_setup else metadata['data']

normalize_data = data_conf["normalize_data"]
normalize_by_features = data_conf["normalize_by_features"]

normalize_by_moving = data_conf["normalize_by_moving"]
normalize_by_scaling = data_conf["normalize_by_scaling"]

batch_size = data_conf["batch_size"]
if not use_training_setup:
    test_batch_size = data_conf["test_batch_size"] if not 'test_batch_size' in config['data'] else config['data']['test_batch_size']
else:
    test_batch_size = data_conf['test_batch_size']

# digit, fashion
data_name = data_conf["data_name"]

In [ ]:
print(test_batch_size)

In [ ]:
training_data_dir = data_conf["data_dir"]
print(data_name, 'from', training_data_dir)

### Models

In [ ]:
model_conf = metadata['models']

sizes = model_conf.get('sizes', None)
n_sizes = len(sizes) if sizes else 0

iterations = model_conf.get('iterations', None)

bias_std = model_conf.get('bias_std', None)

hidden_dims_dict = model_conf.get('hidden_dims_dict', None)

n_models = model_conf.get('n_models', None)

models_total = model_conf.get('models_total', None)

smallest_net_densities = model_conf.get('smallest_net_densities', None)

In [ ]:
param_keys, model_architectures = infer_metada_from_hyperparams(hyperparams)

In [ ]:
param_keys

In [ ]:
print(sizes, iterations, bias_std, hidden_dims_dict, n_models, models_total, smallest_net_densities)

### Training 

In [ ]:
train_config = metadata['models']

evaluation_scheme = train_config.get('evaluation_scheme', [])

In [ ]:
print(evaluation_scheme, len(evaluation_scheme))

### Pruning

In [ ]:
pruning_conf = metadata['pruning']

prune_weights = pruning_conf['prune_weights'] # False -> prune nodes
layer_wise_pruning = pruning_conf['layer_wise_pruning'] # False -> global pruning (not implemented for pytorch pruning)

prune_all_layers = pruning_conf['prune_all_layers'] # should also the weights on the output layer be pruned?

random_mask = pruning_conf['random_mask'] # does the random init network have a random mask as well?

xscale = 'linear' # depends how we define the pruning rates, either linear or logit

In [ ]:
print(prune_weights, layer_wise_pruning, prune_all_layers, random_mask)

### Statistics

In [ ]:
if results_from_training:
    stat_conf = metadata['experiments']
else:
    stat_conf = config['experiments']
stat_conf

In [ ]:
# Counting ARs
compute_stats = stat_conf.get('compute_stats', True)

## 2D
classes = stat_conf.get('classes', None)

# how much one class samples should be covered?
class_coverage = stat_conf.get('class_coverage', None)

use_three_samples = not stat_conf.get('plane_through_origin', True)

average_over_images = stat_conf.get('average_over_images', None)

# Mining LPs with Decision Trees
lp_samples = stat_conf.get('lp_samples', None)


arc_layers = {
    'lenet': 1,
    'deepfc': 2,
    'pipefc': 2
}

'''
# Dark neurons
dm_limit_perc = 0.01
dark_mask_limit = int(dm_limit_perc*test_batch_size)
print(f'dark mask flags neurons with n <= {dark_mask_limit} images (out of {test_batch_size})')
'''

print(compute_stats, classes, class_coverage, use_three_samples, average_over_images, lp_samples)

## Help functions for saving

In [ ]:
ts = datetime.now().strftime('%y%m%d-%H%M%S')

In [ ]:
folder_name = f'visualizations-{ts}-{test_name}/'
saving_folder = results_dir + folder_name
if (save_figures) and not os.path.isdir(saving_folder):
    os.mkdir(saving_folder)
    print('created folder for saving figures')

if save_figures:
    print(f'save images & data to {saving_folder}')
else:
    print("don't save figures")

In [ ]:
data.keys()

In [ ]:
name = list(hyperparams.keys())[0]

In [ ]:
1 - hyperparams[name][-1][0][1]

In [ ]:
print(f'each set of hyperparams has {n_models} models')
print()
print_hyperparams(hyperparams)

# Visualize

1) Accuracy  
2) Activation Pattern based
    - Unique activation patterns
    - Unique layer patterns
    - Entropy
    - Purity
3) Changes in weights  
4) Number of Activation Regions (Boris & David)  
5) Weight distributions  
6) #AR by iteration
    - For the network as a whole
    - By layer
7) Dark Neurons  
    - How many horizons the datapoints were over?  
    - Distribution of Horizons  
    - Dark neuron movements  
10) AR distributions

### Accuracy


In [ ]:
acc_data = data['accuracy'] if not results_from_training else data['acc_training']

In [ ]:
if not results_from_training:
    sparse_from = model_architectures[0]
    # for prese
    plt.figure(figsize=(8,6), dpi=250).patch.set_facecolor('w')
    accs = np.array(acc_data['trained'][sparse_from][-2])
    densities = parse_densities_from_list_of_hyperparams(hyperparams[sparse_from][-2])

    plt.plot(densities, accs[:,0], marker='o', markerfacecolor='k', c='b')
    plt.fill_between(densities, accs[:,1], accs[:,2], color='b', alpha=0.1)

    plt.xlabel('density')
    plt.ylabel('val accuracy (%)')

    plt.title('Validation accuracy of 4 networks with 9.5k parameters each')

    plt.show()

#### Comparing the smallest models to largest models, and most sparse to most dense (large) 

In [ ]:
if not results_from_training:
    print('--- SD to LS ---')
    small_param_acc = np.array(acc_data['trained'][name][-1])[:,0]
    print(small_param_acc)
    print(round((small_param_acc[0] - small_param_acc[-1])/100, 4))
    
    print('--- LS to LD ---')
    large_param_acc = np.array(acc_data['trained'][name][0])[:,0]
    print(large_param_acc)
    print(round((large_param_acc[0] - small_param_acc[0])/100, 4))
    
    print('--- SD to LD ---')
    print(round((large_param_acc[0] - small_param_acc[-1])/100, 4))


In [ ]:
if not results_from_training:
    for scenario in acc_data:
        for name in acc_data[scenario]:

            def visualize_acc_hyperp(over_neurons, ylim, auto_ylim):
                onstr = 'over-neurons' if over_neurons else 'over-densities'
                ystr = f'ylim_{ylim}' if auto_ylim else 'ylim_auto'
                scstr = scenario if not 'trained' in scenario else f'{scenario} {iterations} iterations'

                draw_validation_accuracy(acc_data[scenario],
                                         name,
                                         scstr,
                                         hyperparams,
                                         over_neurons,
                                         filename=saving_folder +
                                         f'accuracy_{scenario}_{name}_{onstr}_{ystr}.png',
                                         ylim=ylim,
                                         auto_ylim=auto_ylim,
                                         save_figures=save_figures)

            over_n = [False, True]
            if data_name == 'fashion':
                ylims = [(50,100), (82,94)] if scenario == 'trained' and name == 'lenet' else [(0,100)]
                ylims = [(50,100), (72,94)] if scenario == 'trained' and name == 'deepfc' else ylims
                auto_ylims = [False, True] if scenario == 'trained' else [False]
            else:
                ylims = [(50,100), (90,100)] if scenario == 'trained' else [(0,100)]
                auto_ylims = [False, True] if scenario == 'trained' else [False]


            for over_neurons in over_n:
                for i, ylim in enumerate(ylims):
                    visualize_acc_hyperp(over_neurons, ylim, auto_ylims[i])

In [ ]:
if results_from_training:
    sparse_from='lenet'
    xscale = 'linear'
    filename = saving_folder + f'accuracy-{sparse_from}_' + 'p{}' + f'_{xscale}.png'
    for sparse_from in acc_data:
        draw_validation_accuracy_from_training(acc_data, sparse_from, evaluation_scheme,
                                               hyperparams_for_sparse=hyperparams,
                                               n_models=n_models,
                                               xscale=xscale,
                                              filename=filename,
                                              save_figures=save_figures)

    filename = saving_folder + f'accuracy-{sparse_from}_' + 'p{}' + f'_{xscale}_xlim100.png'
    for sparse_from in acc_data:
        draw_validation_accuracy_from_training(acc_data, sparse_from,
                                               evaluation_scheme,
                                               hyperparams_for_sparse=hyperparams,
                                               n_models=n_models,
                                               xscale=xscale,
                                               xlim=(-1,100),
                                              filename=filename,
                                              save_figures=save_figures)

    xscale = 'log'
    filename = saving_folder + f'accuracy-{sparse_from}_' + 'p{}' + f'_{xscale}.png'
    for sparse_from in acc_data:
        draw_validation_accuracy_from_training(acc_data, sparse_from, evaluation_scheme,
                                               hyperparams_for_sparse=hyperparams,
                                               n_models=n_models,
                                               xscale=xscale,
                                              filename=filename,
                                              save_figures=save_figures)


In [ ]:
draw_hyperparam_setup(hyperparams, filename=saving_folder + 'hyperparameter_setup.png',
                      save_figures=save_figures)

## Network specialization

In [ ]:
if not results_from_training:
    specialization_data = data['specialization']
    print(specialization_data.keys())
elif compute_stats:
    # now the specialization data has shape
    # architecture: params: [#hyperp, #models, #evaluations, #subspaces, #coverages, #metrics]
    #   e.g. lenet: 120000: (1, 3, 28, 5, 4, 23)
    specialization_data = data['spec_training']
else:
    specialization_data = None

In [ ]:
if specialization_data is not None:
    pre_aggregated = specialization_data['trained'][name][0].shape[-1] == 3
    print('data is pre-aggregated ', pre_aggregated)

In [ ]:
class_coverage

In [ ]:
if not results_from_training:
    large_param_spec = np.mean(specialization_data['trained'][name][0][:,:,0,1,0])
    small_param_spec = np.mean(specialization_data['trained'][name][-1][:,:,0,1,0], axis=1)
    print(large_param_spec.shape, small_param_spec.shape)
    print(round(large_param_spec*100, 2), small_param_spec*100)
    print('SD to LS', round(small_param_spec[0] - small_param_spec[-1],4))
    print('SD to LD', round(large_param_spec - small_param_spec[-1],4))
    print('LS to LD', round(large_param_spec - small_param_spec[0],4))

In [ ]:
if not results_from_training:

    sparse_from = 'lenet'

    metrics_s = [
        'specialization ratio',
        'total area classes',
        'local area'
    ]

    metrics_ac = [f'area of class {i}' for i in range(10)]
    
    metrics_sc = [f'specialization of class {i}' for i in range(10)]

    metrics_spec_all = metrics_s + metrics_ac
    
    if not pre_aggregated:
        metrics_spec_all += metrics_sc

    metrics_spec_dict = {
        m: i for i, m in enumerate(metrics_spec_all)
    }

    print(metrics_spec_dict)

In [ ]:
def get_spec_filename_end(sparse_from, wide_interval, draw_init, over_neurons, class_cov_i):
    distr = '_with-init' if draw_init else ''
    wistr = 'wide' if wide_interval else 'narrow'
    orstr = 'over-neurons' if over_neurons else 'over-density'
    ccstr = f'_cc{round(class_coverage[class_cov_i]*1000)}' if class_cov_i else ''
    return f'{sparse_from}_{orstr}_{wistr}{distr}{ccstr}.png'

In [ ]:
if type(class_coverage) == list:
    class_cov_i = 1
    print(f'Use class coverage {class_coverage[class_cov_i]} as the default')
else:
    print(f'class coverage {class_coverage}')

In [ ]:
if not results_from_training:
    
    draw_method = draw_aggregated_specialization_data if pre_aggregated else draw_specialization_data

    for name in specialization_data[list(specialization_data.keys())[0]]:

        def visualize_specialization_hyperp(plot_name, metrics, name, wide_interval, draw_init, over_neurons, class_cov_i=None):
            file_stub = f'specialization_data_{plot_name}_{get_spec_filename_end(name, wide_interval, draw_init, over_neurons, class_cov_i)}'
            draw_method(specialization_data,
                        metrics,
                        name,
                        over_neurons,
                        param_keys,
                        lp_samples,
                        hyperparams,
                        wide_interval,
                        arc_layers,
                        metrics_spec_dict,
                        average_over_images,
                        use_three_samples,
                        class_cov_i=class_cov_i,
                        class_coverages=class_coverage,
                        draw_init=draw_init,
                        xscale=xscale,
                        filename=saving_folder + file_stub,
                        save_figures=save_figures)

        if type(class_coverage) == list:
            
            # single specialization figures over density
            for cc_i in range(len(class_coverage)):
                plot_name = '1_specialmeasure'
                metrics = [metrics_s[0]] # specialization
                wide_interval = False
                draw_init = False
                over_neurons = False
                visualize_specialization_hyperp(plot_name, metrics, name, wide_interval, draw_init, over_neurons, cc_i)
                
            # single specialization figures over neurons
            for cc_i in range(len(class_coverage)):
                plot_name = '1_specialmeasure_overn'
                over_neurons = True
                visualize_specialization_hyperp(plot_name, metrics, name, wide_interval, draw_init, over_neurons, cc_i)

        else:
            plot_name = '1_specialmeasure'
            metrics = [metrics_s[0]]
            wide_interval = False
            draw_init = False
            over_neurons = False
            visualize_specialization_hyperp(plot_name, metrics, name, wide_interval, draw_init, over_neurons, class_cov_i)
            
            plot_name = '1_specialmeasure_overn'
            over_neurons = True
            visualize_specialization_hyperp(plot_name, metrics, name, wide_interval, draw_init, over_neurons, class_cov_i)
            
        plt.show()

        plot_name = ''
        metrics = metrics_s
        wide_interval = False
        draw_init = False
        over_neurons = False
        visualize_specialization_hyperp(plot_name, metrics, name, wide_interval, draw_init, over_neurons, class_cov_i)

        wide_interval = True
        draw_init = False
        over_neurons = False
        visualize_specialization_hyperp(plot_name, metrics, name, wide_interval, draw_init, over_neurons, class_cov_i)

        wide_interval = False
        draw_init = True
        over_neurons = False
        visualize_specialization_hyperp(plot_name, metrics, name, wide_interval, draw_init, over_neurons, class_cov_i)

        wide_interval = False
        draw_init = True
        over_neurons = True
        visualize_specialization_hyperp(plot_name, metrics, name, wide_interval, draw_init, over_neurons, class_cov_i)

        plot_name = 'spec_classes' if not pre_aggregated else 'area_classes'
        metrics = metrics_sc if not pre_aggregated else metrics_ac
        wide_interval = False
        draw_init = False
        over_neurons = False
        visualize_specialization_hyperp(plot_name, metrics, name, wide_interval, draw_init, over_neurons, class_cov_i)

        if 'init' in specialization_data.keys():
            wide_interval = False
            draw_init = True
            over_neurons = False
            visualize_specialization_hyperp(plot_name, metrics, name, wide_interval, draw_init, over_neurons, class_cov_i)


#### Accuracy as the function of specialization 

In [ ]:
scenario = 'trained'
sparse_from = name

if not results_from_training or compute_stats:

    for cov_i in list(range(len(class_coverage))):
        aggregated_spec = specialization_data[scenario][sparse_from][0].shape[-1] == 3 # if the data is already aggregated, the last values are ave,min,max.

        filename = saving_folder + 'accuracy_over_spec_' + scenario + '_cov_' + str(round(class_coverage[cov_i]*1000)) + '.png'

        draw_validation_accuracy_over_specialization(acc_data[scenario], 
                                                     specialization_data,
                                                     cov_i=cov_i,
                                                     coverages=class_coverage,
                                                     sparse_from=sparse_from,
                                                     scenario=scenario,
                                                     hyperparams=hyperparams,
                                                     auto_ylim=True,
                                                     aggregated_spec=aggregated_spec,
                                                     filename=filename,
                                                     save_figures=save_figures,
                                                    )

#### Specialization as a function of minimum coverage 

In [ ]:
def get_filename_spec_over_cc(scenario, draw_init=False):
    instr = '_with_init' if draw_init and 'init' not in scenario else ''
    filename = saving_folder + 'specialization_over_min_cov_p{}_' + scenario + instr + '.png'
    return filename


if not results_from_training:
    draw_method = draw_specialization_data_over_min_coverage if not pre_aggregated else draw_aggregated_specialization_data_over_min_coverage
    draw_init = False

    if 'init' in list(specialization_data.keys()):
        scenario = 'init'
        linestyles = ['--']
        filename = get_filename_spec_over_cc(scenario, draw_init)

        draw_method(specialization_data, 
                    name, 
                    class_coverage, 
                    param_keys, 
                    lp_samples, 
                    hyperparams=hyperparams,
                    draw_init=draw_init,
                    average_over_images=average_over_images, 
                    use_three_samples=use_three_samples, 
                    linestyles=linestyles,
                    scenario=scenario,
                    filename=filename,
                    save_figures=save_figures)
    
    scenario = 'trained'
    linestyles = ['-']
    filename = get_filename_spec_over_cc(scenario, draw_init)

    draw_method(specialization_data, 
                name, 
                class_coverage, 
                param_keys, 
                lp_samples, 
                hyperparams=hyperparams,
                draw_init=draw_init,
                average_over_images=average_over_images, 
                use_three_samples=use_three_samples, 
                linestyles=linestyles,
                scenario=scenario,
                filename=filename,
                save_figures=save_figures)

In [ ]:
def get_filename_spec_over_iter(cov_i, xscale, xlim=None):
    limstr = f'_xlim{xlim[1]}' if xlim else ''
    cov_str = str([class_coverage[i] for i in cov_i]).replace(' ', '')
    print('use coverages', cov_str)
    
    return saving_folder+'spec_training_covs_' + cov_str + '_p{}_' + xscale + limstr + '.png'

    
    
if results_from_training and compute_stats:
    different_cov_sets = [[0], [0,-1], list(range(len(class_coverage)))]
    
    for cov_i in different_cov_sets:

        xscale='log'
        filename= get_filename_spec_over_iter(cov_i, xscale, xlim=None)

        draw_specialization_from_training(specialization_data,
                                        'lenet',
                                        hyperparams,
                                        cov_i,
                                        class_coverage,
                                        xscale,
                                        evaluation_scheme,
                                        filename=filename,
                                        save_figures=save_figures)

        xscale='linear'
        filename = get_filename_spec_over_iter(cov_i, xscale, xlim=None)
        draw_specialization_from_training(specialization_data,
                                        'lenet',
                                        hyperparams,
                                        cov_i,
                                        class_coverage,
                                        xscale,
                                        evaluation_scheme,
                                        filename=filename,
                                        save_figures=save_figures)

        xscale='linear'
        xlim = (-1,100)
        filename=get_filename_spec_over_iter(cov_i, xscale, xlim=xlim)
            
        draw_specialization_from_training(specialization_data,
                                        'lenet',
                                        hyperparams,
                                        cov_i,
                                        class_coverage,
                                        xscale,
                                        evaluation_scheme,
                                        xlim=xlim,
                                        filename=filename,
                                        save_figures=save_figures)

## Layer pattern mining

In [ ]:
if not results_from_training:

    tree_data = data['trees']
    tree_data.keys()

In [ ]:
if not results_from_training:

    sparse_from = 'lenet'

    metrics_tree = [
        'tree depth',
        'n leaves',
        'average leaf depth', 
        f'useful neurons out of layer {arc_layers[sparse_from]+1} neurons'
    ]
    metrics_sc = [f'max support class {i}' for i in range(10)]
    metrics_pc = [f'number of patterns for class {i}' for i in range(10)]
    metrics_all = metrics_tree + metrics_sc + metrics_pc

    metrics_dict = {
        m: i for i, m in enumerate(metrics_all)

    }

    metrics_dict

In [ ]:
def parse_densities_from_list_of_hyperparams(hyperp):
    return [hyperp[i][1] for i in range(len(hyperp))]


In [ ]:
def get_tree_filename_end(sparse_from, over_neurons, draw_init):
    distr = 'with-init' if draw_init else 'without-init'
    onstr = 'over-neurons' if over_neurons else 'over-density'
    return f'{sparse_from}_{onstr}_{distr}.png'

In [ ]:
if not results_from_training:

    for name in tree_data[list(tree_data.keys())[0]]:
        def visualize_tree_hyperp(plot_name, name, metrics, over_neurons, draw_init):
            filename = f'tree_data_{plot_name}_{get_tree_filename_end(name, over_neurons, draw_init)}'
            draw_tree_data_constant_param(tree_data, metrics, name, over_neurons=over_neurons,
                                  param_keys=param_keys,
                                   lp_samples=lp_samples,
                                  hyperparams_for_sparse=hyperparams,
                                   arc_layers=arc_layers,
                                   metrics_dict=metrics_dict,
                                  draw_init=draw_init,
                                  filename=saving_folder + filename,
                                  save_figures=save_figures)

        plot_name = ''
        metrics = metrics_tree
        over_neurons = False
        draw_init = False
        visualize_tree_hyperp(plot_name, name, metrics, over_neurons, draw_init)

        over_neurons = True
        draw_init = True
        visualize_tree_hyperp(plot_name, name, metrics, over_neurons, draw_init)

        plot_name = 'max-support'
        metrics = metrics_sc
        over_neurons = False
        draw_init = False
        visualize_tree_hyperp(plot_name, name, metrics, over_neurons, draw_init)

        over_neurons = True
        draw_init = True
        visualize_tree_hyperp(plot_name, name, metrics, over_neurons, draw_init)

        plot_name = 'unique_patterns'
        metrics = metrics_pc
        over_neurons = False
        draw_init = False
        visualize_tree_hyperp(plot_name, name, metrics, over_neurons, draw_init)

        over_neurons = True
        draw_init = True
        visualize_tree_hyperp(plot_name, name, metrics, over_neurons, draw_init)



## Unique activation patterns

In [ ]:
if not results_from_training:
    uaps = data['uaps']

In [ ]:
if compute_stats and visualize_AP_based and not results_from_training:
    for sparse_from in uaps[list(uaps.keys())[0]]:
        draw_unique_aps(uaps, sparse_from,
                       test_batch_size,
                        hyperparams_for_sparse=hyperparams,
                       blacklist=[],
                       filename=saving_folder+f'unique-aps-{sparse_from}.png',
                       xscale=xscale,
                       save_figures=save_figures,
                       auto_yscale=False)

## Unique layer patterns

In [ ]:
if not results_from_training:
    ulps = data['ulps']

In [ ]:
sparse_from = name

if compute_stats and visualize_AP_based and not results_from_training:
    for sparse_from in ulps[list(ulps.keys())[0]]:

        draw_unique_lps(ulps, sparse_from, test_batch_size,
                        hyperparams,
                        blacklist=[],
                       filename=saving_folder + f'unique_layer_patterns_{sparse_from}.png',
                       xscale=xscale,
                       save_figures=save_figures,
                       auto_yscale=True)

## Entropy

In [ ]:
if not results_from_training:
    entropy_stats = data['entropy_stats']

In [ ]:
if compute_stats and visualize_AP_based and not results_from_training:
    for sparse_from in entropy_stats[list(entropy_stats.keys())[0]]:

        draw_entropy(entropy_stats, sparse_from, test_batch_size, hyperparams,
                         blacklist=[],
                         filename=saving_folder + f'entropy-{sparse_from}.png',
                         save_figures=save_figures,
                         xscale=xscale)

## Purity

In [ ]:
pass

## Number of 2D ARs

In [ ]:
ar_data_2D = data['ar_2d'] if not results_from_training else data['ar_2d_training']

In [ ]:
if not results_from_training:
    pre_aggregated = ar_data_2D['trained'][name][0].shape[-1] == 3
    print('data is pre-aggregated ', pre_aggregated)

In [ ]:
if not results_from_training:
    draw_method = draw_number_of_2D_ARs_aggregated if pre_aggregated else draw_number_of_2D_ARs

    def visualize_2d_ars_hyperp(sparse_from, wide_interval, normalized_by_area, just_area, over_neurons, draw_init):
        wstr = 'wide' if wide_interval else 'narrow'
        astr = 'normalized_by_area' if normalized_by_area else 'absolute_numbers'
        arstr = 'just_area' if just_area else 'regions'
        onstr = 'over_neurons' if over_neurons else 'over_densities'
        istr = '_winit' if draw_init else ''

        filename =  f'activation_regions_2D_{arstr}_{sparse_from}_{astr}_{wstr}_{onstr}_{istr}.png'
        draw_method(ar_data_2D,
                    sparse_from,
                    normalized_by_area,
                    average_over_images, 
                    hyperparams,
                    param_keys=param_keys,
                    over_neurons=over_neurons,
                    n_models=n_models,
                    wide_interval=wide_interval,
                    just_area=just_area,
                    draw_init=draw_init,
                    use_three_samples=use_three_samples, # if we have different scenarios (planes through origin and spun by 3 imgs) set this to None
                    filename=saving_folder + filename,
                    save_figures=save_figures)

    for sparse_from in ar_data_2D[list(ar_data_2D.keys())[0]]:
            over_neurons=False
            draw_init = False
            just_area = False
            wide_interval = False
            normalized_by_area = True
            visualize_2d_ars_hyperp(sparse_from, wide_interval, normalized_by_area, just_area, over_neurons, draw_init)

            normalized_by_area = False
            visualize_2d_ars_hyperp(sparse_from, wide_interval, normalized_by_area, just_area, over_neurons, draw_init)

            over_neurons=True
            normalized_by_area = True
            visualize_2d_ars_hyperp(sparse_from, wide_interval, normalized_by_area, just_area, over_neurons, draw_init)
            
            normalized_by_area = False
            visualize_2d_ars_hyperp(sparse_from, wide_interval, normalized_by_area, just_area, over_neurons, draw_init)
            
            draw_init = True
            visualize_2d_ars_hyperp(sparse_from, wide_interval, normalized_by_area, just_area, over_neurons, draw_init)

            draw_init = False
            wide_interval = True
            normalized_by_area = False
            visualize_2d_ars_hyperp(sparse_from, wide_interval, normalized_by_area, just_area, over_neurons, draw_init)

            just_area = True
            wide_interval = True
            visualize_2d_ars_hyperp(sparse_from, wide_interval, normalized_by_area, just_area, over_neurons, draw_init)



## #AR by iteration

In [ ]:
def get_ar_by_iteration_filename(sparse_from, xlim, wide_interval, normalized_by_area, xscale):
    wstr = 'wide' if wide_interval else 'narrow'
    astr = 'normalized_by_area' if normalized_by_area else 'absolute_numbers'
    limstr = 'xlim100' if xlim else ''

    return saving_folder + f'ARs_by_iteration_2D_{sparse_from}_{astr}_{wstr}_' + 'p{}' + f'_{xscale}_{limstr}.png'

if results_from_training and compute_stats:

    sparse_from = 'lenet'
    normalized_by_area = False
    wide_interval = False
    xscale = 'log'
    
    filename = get_ar_by_iteration_filename(sparse_from, False, wide_interval, normalized_by_area, xscale)
    
    draw_number_of_2D_ARs_by_iteration(ar_data_2D, 
                                       sparse_from, 
                                       normalized_by_area,
                                       use_three_samples,
                                      average_over_images, 
                                       evaluation_scheme,
                                       hyperparams_for_sparse=hyperparams,
                                       n_models=n_models,
                                      wide_interval=wide_interval,
                                       xscale=xscale,
                                      filename=filename,
                                      save_figures=save_figures)
    
    xscale = 'linear'
    filename = get_ar_by_iteration_filename(sparse_from, False, wide_interval, normalized_by_area, xscale)
    
    draw_number_of_2D_ARs_by_iteration(ar_data_2D, 
                                       sparse_from, 
                                       normalized_by_area,
                                       use_three_samples,
                                      average_over_images, 
                                       evaluation_scheme,
                                       hyperparams_for_sparse=hyperparams,
                                       n_models=n_models,
                                      wide_interval=wide_interval,
                                       xscale=xscale,
                                      filename=filename,
                                      save_figures=save_figures)
    
    filename = get_ar_by_iteration_filename(sparse_from, True, wide_interval, normalized_by_area, xscale)
    
    draw_number_of_2D_ARs_by_iteration(ar_data_2D, 
                                       sparse_from, 
                                       normalized_by_area,
                                       use_three_samples,
                                      average_over_images, 
                                       evaluation_scheme,
                                       hyperparams_for_sparse=hyperparams,
                                       n_models=n_models,
                                      wide_interval=wide_interval,
                                       xscale=xscale,
                                       xlim=(-1,100),
                                      filename=filename,
                                      save_figures=save_figures)